### supp figure: cross mappale clusters

In [ ]:
clusters_annotated = load_across_tissues(config, load_clusters_annotated)
sns.histplot(clusters_annotated, x='num_cross_map', hue='has_cross_map', bins=np.arange(0, clusters_annotated['num_cross_map'].max()))